# Model Monitoring

In [1]:
# Import libraries
from datetime import datetime, timedelta, timezone
import json
import os
import re
import boto3
import uuid
import time
from threading import Thread
import pandas as pd
from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer
from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig
from sagemaker.predictor import Predictor
from time import gmtime, strftime, sleep
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor import EndpointInput
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.model_monitor import CronExpressionGenerator
from tqdm.notebook import tqdm
from sagemaker.model_monitor import DefaultModelMonitor

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
# Create CloudWatch client
cw_client = boto3.Session().client("cloudwatch")
namespace = "aws/sagemaker/Endpoints/model-metrics"

In [3]:
# Setup boto and sagemaker session
sagemaker_session = Session()
role = get_execution_role()
region = sagemaker_session.boto_region_name

# Setup S3 bucket
bucket = sagemaker_session.default_bucket()
print("Bucket:", bucket)
prefix = f"sagemaker/FoodLens-ModelQualityMonitor-{datetime.now():%Y-%m-%d-%H-%M-%S}"

# S3 prefixes
data_capture_prefix = f"{prefix}/datacapture"
s3_capture_upload_path = f"s3://{bucket}/{data_capture_prefix}"

ground_truth_upload_path = (
    f"s3://{bucket}/{prefix}/ground_truth_data/{datetime.now():%Y-%m-%d-%H-%M-%S}"
)
%store ground_truth_upload_path

reports_prefix = f"{prefix}/reports"
s3_report_path = f"s3://{bucket}/{reports_prefix}"

print(f"Capture path: {s3_capture_upload_path}")
print(f"Ground truth path: {ground_truth_upload_path}")
print(f"Report path: {s3_report_path}")

Bucket: sagemaker-us-east-1-654654380268
Stored 'ground_truth_upload_path' (str)
Capture path: s3://sagemaker-us-east-1-654654380268/sagemaker/FoodLens-ModelQualityMonitor-2025-10-17-20-37-43/datacapture
Ground truth path: s3://sagemaker-us-east-1-654654380268/sagemaker/FoodLens-ModelQualityMonitor-2025-10-17-20-37-43/ground_truth_data/2025-10-17-20-37-43
Report path: s3://sagemaker-us-east-1-654654380268/sagemaker/FoodLens-ModelQualityMonitor-2025-10-17-20-37-43/reports


## Deploy Pre-Trained Model to Live Endpoint

In [4]:
# Initialize the sagemaker client
sagemaker_client = boto3.client("sagemaker")

# Specify model
image_uri = image_uris.retrieve(framework="xgboost", region=region, version="1.7-1")
instance_type = 'ml.m5.xlarge'
# model_name = 'nutrition-score-xgb-2025-10-17-13-03-05' # get from notebook 04
%store -r model_name
response = sagemaker_client.describe_model(ModelName=model_name)
model_url = response['PrimaryContainer']['ModelDataUrl']
model = Model(image_uri=image_uri, model_data=model_url, role=role, sagemaker_session=sagemaker_session)

In [5]:
endpoint_name = f"xgb-nutriscore-monitor-{datetime.now():%Y-%m-%d-%H-%M-%S}"
%store endpoint_name
print("EndpointName: ", endpoint_name)

# Enable data capture
data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

# Deploy the model and wait for it to be in service
print("Deploying endpoint....")
model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

print(f"\nEndpoint '{endpoint_name}' in Service.")

Stored 'endpoint_name' (str)
EndpointName:  xgb-nutriscore-monitor-2025-10-17-20-37-47
Deploying endpoint....
------!
Endpoint 'xgb-nutriscore-monitor-2025-10-17-20-37-47' in Service.


In [7]:
# Create predictor object
predictor = Predictor(
    endpoint_name=endpoint_name, 
    sagemaker_session=sagemaker_session, 
    serializer=CSVSerializer()
)

## Setup Infrastructure Monitoring

In [8]:
# Create a CloudWatch alarm for model latency
alarm_name = "NUTRISCORE_MODEL_LATENCY_HIGH"
alarm_desc = "Trigger an alarm when the average model latency exceeds 300ms."
infrastructure_metric_name = 'ModelLatency'
cw_infrastructure_dimensions = [
    {"Name": "Endpoint", "Value": endpoint_name},
    {"Name": "MonitoringSchedule", "Value": 'AllTraffic'},
]

# Create the alarm
cw_client.put_metric_alarm(
    AlarmName=alarm_name,
    AlarmDescription=alarm_desc,
    ActionsEnabled=False,  # Change to True if you want notifications
    MetricName=infrastructure_metric_name,
    Namespace="AWS/SageMaker",
    Statistic="Average",
    Dimensions=cw_infrastructure_dimensions,
    Period=300,  # check every 5 minutes
    EvaluationPeriods=1,
    Threshold=300.0,  # 300 milliseconds threshold
    ComparisonOperator="GreaterThanThreshold",
    TreatMissingData="missing",
)

print("Successfully created alarm: NUTRISCORE_MODEL_LATENCY_HIGH")

Successfully created alarm: NUTRISCORE_MODEL_LATENCY_HIGH


In [9]:
# Alarm for 5xx Server Errors
cw_client.put_metric_alarm(
    AlarmName="NUTRISCORE_MODEL_5XX_ERRORS_HIGH",
    AlarmDescription="Trigger an alarm when any 5xx server-side error occurs.",
    ActionsEnabled=False,
    MetricName="Invocation5XXErrors",
    Namespace="AWS/SageMaker",
    Statistic="Sum",
    Dimensions=cw_infrastructure_dimensions,
    Period=300,  # 5 minutes
    EvaluationPeriods=1,
    Threshold=0, # Alarm if even one error (Sum > 0) occurs
    ComparisonOperator="GreaterThanThreshold",
    TreatMissingData="notBreaching",
)
print("Successfully created alarm: NUTRISCORE_MODEL_5XX_ERRORS_HIGH")

Successfully created alarm: NUTRISCORE_MODEL_5XX_ERRORS_HIGH


In [10]:
# Alarm for High CPU Utilization
cw_client.put_metric_alarm(
    AlarmName="NUTRISCORE_MODEL_CPU_HIGH",
    AlarmDescription="Trigger an alarm when average CPU utilization exceeds 80%.",
    ActionsEnabled=False,
    MetricName="CPUUtilization",
    Namespace="AWS/SageMaker",
    Statistic="Average",
    Dimensions=cw_infrastructure_dimensions,
    Period=300,  # 5 minutes
    EvaluationPeriods=1,
    Threshold=80.0, # 80 percent
    ComparisonOperator="GreaterThanThreshold",
    TreatMissingData="missing", # 'missing' means don't alarm if there's no traffic
)
print("Successfully created alarm: NUTRISCORE_MODEL_CPU_HIGH")

Successfully created alarm: NUTRISCORE_MODEL_CPU_HIGH


## Setup Data Quality Monitor

In [51]:
# Build baseline from scaled training data
# From Notebook 04, train data with headers
train_s3_path = f's3://{bucket}/nutriscore-prediction-xgboost/train/train_scaled_headers_features_only.csv'

# The S3 path where the data quality reports will be stored
data_quality_report_path = f"s3://{bucket}/nutriscore-prediction-xgboost/data-quality-reports"

# Create a Data Quality Monitor object
data_quality_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type=instance_type,
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
    sagemaker_session=sagemaker_session,
)

data_quality_baseline_job_name = f"nutriscore-data-quality-baseline-job-{datetime.now():%Y-%m-%d-%H-%M-%S}"

print("Starting Data Quality baseline suggestion job...")
# The baseline job runs on the training data
data_quality_monitor.suggest_baseline(
    baseline_dataset=train_s3_path,
    dataset_format=DatasetFormat.csv(header=True),
    job_name=data_quality_baseline_job_name,
    output_s3_uri=data_quality_report_path,
    wait=True,
    logs=False,
)
print("\nData Quality baseline job complete.")

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating processing-job with name nutriscore-data-quality-baseline-job-2025-10-17-23-15-15


Starting Data Quality baseline suggestion job...
...........................................................!
Data Quality baseline job complete.


In [52]:
data_quality_monitor.latest_baselining_job.describe()

{'ProcessingInputs': [{'InputName': 'baseline_dataset_input',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-654654380268/nutriscore-prediction-xgboost/train/train_scaled_headers_features_only.csv',
    'LocalPath': '/opt/ml/processing/input/baseline_dataset_input',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'monitoring_output',
    'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-654654380268/nutriscore-prediction-xgboost/data-quality-reports',
     'LocalPath': '/opt/ml/processing/output',
     'S3UploadMode': 'EndOfJob'},
    'AppManaged': False}]},
 'ProcessingJobName': 'nutriscore-data-quality-baseline-job-2025-10-17-23-15-15',
 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 1,
   'InstanceType': 'ml.m5.xlarge',
   'VolumeSizeInGB': 20}},
 'StoppingCondition': {'MaxRuntimeInSeconds': 3600},
 

In [53]:
# Create data quality monitoring schedule
data_quality_schedule_name = f"nutriscore-data-quality-schedule-{datetime.now():%Y-%m-%d-%H-%M-%S}"

print(f"Creating Data Quality monitoring schedule: {data_quality_schedule_name}")
data_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=data_quality_schedule_name,
    endpoint_input=predictor.endpoint_name,
    output_s3_uri=data_quality_report_path,
    statistics=data_quality_monitor.latest_baselining_job.baseline_statistics(),
    constraints=data_quality_monitor.latest_baselining_job.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)
print("Data Quality monitoring schedule created.")

Creating Data Quality monitoring schedule: nutriscore-data-quality-schedule-2025-10-17-23-21-22


INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: nutriscore-data-quality-schedule-2025-10-17-23-21-22


Data Quality monitoring schedule created.


In [54]:
# Create alarm for data quality drift
alarm_name = "NUTRISCORE_DATA_DRIFT_VIOLATIONS"
alarm_desc = "Trigger an alarm when any feature's statistics drift from the baseline."

# Define the dimensions for the Data Quality monitor
cw_data_quality_dimensions = [
    {"Name": "EndpointName", "Value": endpoint_name},
    {"Name": "MonitoringSchedule", "Value": data_quality_schedule_name},
]

# Create the alarm
cw_client.put_metric_alarm(
    AlarmName=alarm_name,
    AlarmDescription=alarm_desc,
    ActionsEnabled=False,
    MetricName="feature_baseline_drift_total_violations",
    Namespace="aws/sagemaker/Endpoints/data-metrics",
    Statistic="Sum",
    Dimensions=cw_data_quality_dimensions,
    Period=3600,  # hourly
    EvaluationPeriods=1,
    Threshold=0, # Alarm if the total number of violations is greater than 0
    ComparisonOperator="GreaterThanThreshold",
    TreatMissingData="missing", # Don't alarm if the job hasn't run yet
)
print(f"Successfully created alarm: {alarm_name}")

Successfully created alarm: NUTRISCORE_DATA_DRIFT_VIOLATIONS


## Generate Baseline for Model Quality Performance

In [15]:
# Get validation dataset
# get from previous notebook
val_s3_path = f's3://{bucket}/nutriscore-prediction-xgboost/validation/val_scaled.csv' 
!aws s3 cp {val_s3_path} ./
val_local_path = './val_scaled.csv'

download: s3://sagemaker-us-east-1-654654380268/nutriscore-prediction-xgboost/validation/val_scaled.csv to ./val_scaled.csv


In [16]:
# Your validation dataset should h first column
limit = 500 # number of samples for baseline
baseline_file_name = 'val_pred_baseline.csv'
i = 0

# Create a new file for your baseline data
with open(f"{baseline_file_name}", "w") as baseline_file:
    # Header for a regression baseline
    baseline_file.write("prediction,label\n")
    
    # Open validation data file
    with open(val_local_path, "r") as f:
        for row in f:
            # With true score in first column
            (label, input_cols) = row.split(",", 1)
            
            # Get the predicted score from the endpoint
            predicted_score = float(predictor.predict(input_cols))
            
            # Write the predicted score and the true label to the baseline file
            baseline_file.write(f"{predicted_score},{label.strip()}\n")
            
            i += 1
            if i >= limit:
                break
            print(".", end="", flush=True)
            sleep(0.5)
print()
print("Done!")

...................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
Done!


In [17]:
# Examine predictions from model
!head {baseline_file_name}

prediction,label
8.639720916748047,9
-3.4511897563934326,-4
25.869300842285156,24
0.5103368759155273,-2
8.74711799621582,8
25.327014923095703,25
29.67618751525879,29
0.3975553512573242,0
9.496760368347168,10


In [18]:
# Upload predictions as baseline dataset
baseline_prefix = prefix + "/baselining"
baseline_data_prefix = baseline_prefix + "/data"
baseline_results_prefix = baseline_prefix + "/results"

baseline_data_uri = f"s3://{bucket}/{baseline_data_prefix}"
baseline_results_uri = f"s3://{bucket}/{baseline_results_prefix}"
print(f"Baseline data uri: {baseline_data_uri}")
print(f"Baseline results uri: {baseline_results_uri}")

Baseline data uri: s3://sagemaker-us-east-1-654654380268/sagemaker/FoodLens-ModelQualityMonitor-2025-10-17-20-37-43/baselining/data
Baseline results uri: s3://sagemaker-us-east-1-654654380268/sagemaker/FoodLens-ModelQualityMonitor-2025-10-17-20-37-43/baselining/results


In [19]:
# Upload baseline dataset
baseline_dataset_uri = S3Uploader.upload(f"{baseline_file_name}", baseline_data_uri)
baseline_dataset_uri

's3://sagemaker-us-east-1-654654380268/sagemaker/FoodLens-ModelQualityMonitor-2025-10-17-20-37-43/baselining/data/val_pred_baseline.csv'

In [20]:
# Create the model quality monitoring object
nutriscore_model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type=instance_type,
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=sagemaker_session,
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [21]:
# Name of the model quality baseline job
baseline_job_name = f"xgb-nutriscore-model-baseline-job-{datetime.now():%Y-%m-%d-%H%M}"

In [22]:
# Execute the baseline suggestion job
job = nutriscore_model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    problem_type="Regression",
    inference_attribute="prediction", # model output
    ground_truth_attribute="label", # true score
)
job.wait(logs=False)

INFO:sagemaker:Creating processing-job with name xgb-nutriscore-model-baseline-job-2025-10-17-2051


...........................................................!

## Explore Results of Baseline Job

In [23]:
baseline_job = nutriscore_model_quality_monitor.latest_baselining_job

In [24]:
# View metrics
binary_metrics = baseline_job.baseline_statistics().body_dict["regression_metrics"]
pd.json_normalize(binary_metrics).T

,0
mae.value,1.065293
mae.standard_deviation,0.027784
mse.value,3.308454
mse.standard_deviation,0.324367
rmse.value,1.818916
rmse.standard_deviation,0.090458
r2.value,0.967103
r2.standard_deviation,0.003188


In [25]:
# View constraints
regression_constraints = pd.DataFrame(baseline_job.suggested_constraints().body_dict["regression_constraints"]).T
regression_constraints

,threshold,comparison_operator
mae,1.065293,GreaterThanThreshold
mse,3.308454,GreaterThanThreshold
rmse,1.818916,GreaterThanThreshold
r2,0.967103,LessThanThreshold


## Create Model Quality Monitoring Schedule

In [26]:
# Set monitor schedule name
nutriscore_model_quality_monitor_schedule_name = f"nutriscore-quality-monitoring-schedule-{datetime.now():%Y-%m-%d-%H-%M-%S}"

In [27]:
# EndpointInput for regression
endpointInput = EndpointInput(
    endpoint_name=predictor.endpoint_name,
    inference_attribute="0", # first column contains inference
    destination="/opt/ml/processing/input_data",
)

In [28]:
# Create the monitoring schedule to execute every hour
response = nutriscore_model_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=nutriscore_model_quality_monitor_schedule_name,
    endpoint_input=endpointInput,
    output_s3_uri=baseline_results_uri,
    problem_type="Regression",
    ground_truth_input=ground_truth_upload_path,
    constraints=baseline_job.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: nutriscore-quality-monitoring-schedule-2025-10-17-20-56-05


In [29]:
# Examine schedule on monitor
nutriscore_model_quality_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:654654380268:monitoring-schedule/nutriscore-quality-monitoring-schedule-2025-10-17-20-56-05',
 'MonitoringScheduleName': 'nutriscore-quality-monitoring-schedule-2025-10-17-20-56-05',
 'MonitoringScheduleStatus': 'Pending',
 'MonitoringType': 'ModelQuality',
 'CreationTime': datetime.datetime(2025, 10, 17, 20, 56, 6, 525000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 10, 17, 20, 56, 6, 604000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'model-quality-job-definition-2025-10-17-20-56-05-802',
  'MonitoringType': 'ModelQuality'},
 'EndpointName': 'xgb-nutriscore-monitor-2025-10-17-20-37-47',
 'ResponseMetadata': {'RequestId': '791bae99-c518-4869-834e-0b970b43a9ee',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '791bae99-c518-4869-834e-0b970b43a9ee',
   'strict-transport-security': 'max-age=47304000

## Setup Model Quality Alarm

In [30]:
# Get thresholds from baseline constraints
rmse_threshold = regression_constraints.loc['rmse', 'threshold']
rmse_operator = regression_constraints.loc['rmse', 'comparison_operator']
print(f"RMSE Threshold from baseline constraints: {rmse_threshold}")
print(f"RMSE Comparison Operator from baseline constraints: {rmse_operator}")

RMSE Threshold from baseline constraints: 1.8189155342515506
RMSE Comparison Operator from baseline constraints: GreaterThanThreshold


In [31]:
# Create a CloudWatch Alarm for a regression metrics
print("Creating CloudWatch alarm for RMSE...")
alarm_name = "NUTRISCORE_MODEL_RMSE_DRIFT"
alarm_desc = "Trigger an alarm when the model's RMSE exceeds the baseline threshold."
cw_quality_dimensions = [
    {"Name": "Endpoint", "Value": endpoint_name},
    {"Name": "MonitoringSchedule", "Value": nutriscore_model_quality_monitor_schedule_name},
]

cw_client.put_metric_alarm(
    AlarmName=alarm_name,
    AlarmDescription=alarm_desc,
    ActionsEnabled=False,
    MetricName="rmse",
    Namespace=namespace,
    Statistic="Average",
    Dimensions=cw_quality_dimensions,
    Period=3600, # Checks every hour
    EvaluationPeriods=1,
    DatapointsToAlarm=1,
    Threshold=rmse_threshold,
    ComparisonOperator=rmse_operator,
    TreatMissingData="breaching",
)
print(f"Successfully created alarm: {alarm_name}")

Creating CloudWatch alarm for RMSE...
Successfully created alarm: NUTRISCORE_MODEL_RMSE_DRIFT


## Continuous Model Monitoring Pre-Processed Prod Data Simulation

In [32]:
# Get some samples from scaled production data split
# From notebook 04
prod_scaled_path = f's3://{bucket}/nutriscore-prediction-xgboost/prod/prod_scaled.csv'
%store prod_scaled_path
prod_scaled_df = pd.read_csv(prod_scaled_path)

Stored 'prod_scaled_path' (str)


In [ ]:
# # Start the simulation in a background thread
# simulation_thread = Thread(
#     target=simulate_live_traffic_for_duration,
#     args=(
#         endpoint_name,
#         sagemaker_session,
#         prod_scaled_df,
#         ground_truth_upload_path,
#         4 # hours duration for simulation
#     )
# )
# simulation_thread.start()

In [39]:
# Initially there will be no executions since the first execution happens at the top of the hour
executions = nutriscore_model_quality_monitor.list_executions()
executions

In [40]:
# Wait for the first execution of the monitoring_schedule
print("Waiting for first execution", end="")
while True:
    execution = nutriscore_model_quality_monitor.describe_schedule().get(
        "LastMonitoringExecutionSummary"
    )
    if execution:
        break
    print(".", end="", flush=True)
    sleep(10)
print()
print("Execution found!")

Waiting for first execution
Execution found!


In [41]:
# View execution details
latest_execution = executions[0]
latest_execution.describe()

{'ProcessingInputs': [{'InputName': 'constraints',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-654654380268/sagemaker/FoodLens-ModelQualityMonitor-2025-10-17-20-37-43/baselining/results/constraints.json',
    'LocalPath': '/opt/ml/processing/baseline/constraints',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated'}},
  {'InputName': 'endpoint_input_1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-654654380268/sagemaker/FoodLens-ModelQualityMonitor-2025-10-17-20-37-43/baselining/results/merge/xgb-nutriscore-monitor-2025-10-17-20-37-47/AllTraffic/2025/10/17/21',
    'LocalPath': '/opt/ml/processing/input_data/xgb-nutriscore-monitor-2025-10-17-20-37-47/AllTraffic/2025/10/17/21',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'resu

In [ ]:
# View execution status
status = execution["MonitoringExecutionStatus"]

while status in ["Pending", "InProgress"]:
    print("Waiting for execution to finish", end="")
    latest_execution.wait(logs=False)
    latest_job = latest_execution.describe()
    print()
    print(f"{latest_job['ProcessingJobName']} job status:", latest_job["ProcessingJobStatus"])
    print(
        f"{latest_job['ProcessingJobName']} job exit message, if any:",
        latest_job.get("ExitMessage"),
    )
    print(
        f"{latest_job['ProcessingJobName']} job failure reason, if any:",
        latest_job.get("FailureReason"),
    )
    sleep(
        30
    )  # model quality executions consist of two Processing jobs, wait for second job to start
    latest_execution = nutriscore_model_quality_monitor.list_executions()[-1]
    execution = nutriscore_model_quality_monitor.describe_schedule()["LastMonitoringExecutionSummary"]
    status = execution["MonitoringExecutionStatus"]

print("Execution status is:", status)

if status != "Completed":
    print(execution)
    print(
        "====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures."
    )

## View Model and Data Reports

In [58]:
# View model report
model_quality_latest_execution = nutriscore_model_quality_monitor.list_executions()[0]
report_uri = model_quality_latest_execution.describe()["ProcessingOutputConfig"]["Outputs"][0]["S3Output"][
    "S3Uri"
]
print("Model Report Uri:", report_uri)

# View violations generated by monitoring schedule
pd.options.display.max_colwidth = None
violations = model_quality_latest_execution.constraint_violations().body_dict["violations"]
violations_df = pd.json_normalize(violations)
violations_df.head(10)

Model Report Uri: s3://sagemaker-us-east-1-654654380268/sagemaker/FoodLens-ModelQualityMonitor-2025-10-17-20-37-43/baselining/results/xgb-nutriscore-monitor-2025-10-17-20-37-47/nutriscore-quality-monitoring-schedule-2025-10-17-20-56-05/2025/10/17/22


""


In [50]:
# View data quality report
data_quality_latest_execution = data_quality_monitor.list_executions()[0]
data_report_uri = data_quality_latest_execution.describe()["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
print("Data Report Uri:", data_report_uri)

# View violations generated by monitoring schedule
pd.options.display.max_colwidth = None
violations = data_quality_latest_execution.constraint_violations().body_dict["violations"]
violations_df = pd.json_normalize(violations)
violations_df.head(10)

Data Report Uri: s3://sagemaker-us-east-1-654654380268/nutriscore-prediction-xgboost/data-quality-reports/xgb-nutriscore-monitor-2025-10-17-20-37-47/nutriscore-data-quality-schedule-2025-10-17-20-46-42/2025/10/17/21


,feature_name,constraint_check_type,description
0,nutriscore_score,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."


In [59]:
# View data quality report
data_quality_latest_execution = data_quality_monitor.list_executions()[0]
data_report_uri = data_quality_latest_execution.describe()["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
print("Data Report Uri:", data_report_uri)

# View violations generated by monitoring schedule
pd.options.display.max_colwidth = None
violations = data_quality_latest_execution.constraint_violations().body_dict["violations"]
violations_df = pd.json_normalize(violations)
violations_df.head(10)

Data Report Uri: s3://sagemaker-us-east-1-654654380268/nutriscore-prediction-xgboost/data-quality-reports/xgb-nutriscore-monitor-2025-10-17-20-37-47/nutriscore-data-quality-schedule-2025-10-17-23-21-22/2025/10/18/00


,feature_name,constraint_check_type,description
0,Extra columns,extra_column_check,"There are extra columns in current dataset. Number of columns in current dataset: 23, Number of columns in baseline constraints: 22"


## Cleanup

In [ ]:
# Delete monitoring schedules
nutriscore_model_quality_monitor.delete_monitoring_schedule()
data_quality_monitor.delete_monitoring_schedule()
predictor.delete_endpoint()